In [1]:
# !pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
# !pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 openai

In [2]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr

### Using Frontier Model

In [3]:
gpt_model = "gpt-4.1"  ### Frontier Model

In [4]:
## Sign in to OpenAI using Secrets in Colab
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [5]:
system_message = "You are an assistant that generate a synthetic dataset using the given instructions"

def user_prompt_message(user_message):
  user_prompt = f"Below is the instruction for the synthetic dataset. {user_message} Please create a synthetic dataset using the given data"
  return user_prompt

In [6]:
def chat_assitance(user_message, history):

  prompt = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": user_prompt_message(user_message)}]

  response = openai.chat.completions.create(
    model = gpt_model,
    messages = prompt,
    stream = True
  )

  result = ""
  for chunk in response:
    result += chunk.choices[0].delta.content or ""
    yield result

In [11]:
gr.ChatInterface(fn = chat_assitance, type = "messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3a1bac6b282838143e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Using Open Source Model

In [7]:
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"  ### Open Source Model

In [8]:
## Sign in to HuggingFace Hub
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [9]:
### Quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [18]:
def model_outputs(user_message, history):
    prompt = [{"role": "system", "content": system_message}] + history + [
        {"role": "user", "content": user_prompt_message(user_message)}
    ]

    LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

    tokenizer = AutoTokenizer.from_pretrained(LLAMA)
    tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer.apply_chat_template(prompt, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)

    model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

    outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)
    response = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)

    yield [{"role": "assistant", "content": response}]

In [19]:
# history = []
# print(model_outputs("Hello, how are you?", history))

In [20]:
gr.ChatInterface(fn = model_outputs, type = "messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://119747bcc849780afa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
